In [ ]:
# Import packages 
import pandas as pd
import numpy as np
import random 
import json

# For getting and processing URDB rates
import os
import certifi
import requests
import json
import time

import PySAM.Utilityrate5 as utility_rate
import PySAM.UtilityRateTools
import PySAM.LoadTools

In [ ]:
# Original agent files. Specify full path to files
agents = pd.read_pickle('../../../data/agent_df_base_res_national_revised_v2.pkl')
rates = pd.read_csv('../../../data/usurdb.csv', dtype={'eiaid':'str'}).rename(columns={'name':'tariff_name', 'eiaid':'eia_id'})

In [ ]:
# Pull tariff name from agents
tariff_names = agents[['eia_id', 'tariff_name']].drop_duplicates().reset_index(drop=True)

# Join label, eiaid, and name to the agent rate name
tariff_names_urdb = tariff_names.merge(
    rates[['eia_id', 'name', ]]
    )

In [ ]:
rates.head()

In [ ]:
for col in rates.columns:
    print(col)

## Code for getting URDB tariff rates ##
This code will return the new tariff rates based on URDB page value. You must have your own URDB API Key to run this code successfully.

In [ ]:
def URDBv8_to_ElectricityRates(urdb_response):
    """
    Formats response from Utility Rate Database API version 8 for use in PySAM
        i.e.
            model = PySAM.UtilityRate5.new()
            rates = PySAM.ResourceTools.URDBv8_to_ElectricityRates(urdb_response)
            model.ElectricityRates.assign(rates)

    This function does the additional processing when these rate features
    are present. 

    :param: urdb_response:
        dictionary with response fields following
        https://openei.org/services/doc/rest/util_rates/?version=8
    :return: dictionary for PySAM.UtilityRate5.UtilityRate5.ElectricityRates
    """
    urdb_data = dict()
    urdb_data['en_electricity_rates'] = 1

    def try_get_schedule(urdb_name, data_name):
        if urdb_name in urdb_response.keys():
            urdb_data[data_name] = urdb_response[urdb_name]
            for i in range(12):
                for j in range(24):
                    urdb_data[data_name][i][j] += 1

    def try_get_rate_structure(urdb_name, data_name):
        mat = []
        supported_units = {
            "kwh" : 0,
            "kwh/kw" : 1,
            "kwh daily" : 2,
            "kwh/kw daily" : 3
        }
        if urdb_name in urdb_response.keys():
            structure = urdb_response[urdb_name]
            for i, period in enumerate(structure):
                for j, entry in enumerate(period):
                    rate = entry['rate']
                    if 'adj' in entry.keys():
                        rate += entry['adj']
                    tier_max = 1e38
                    if 'max' in entry.keys():
                        tier_max = entry['max']
                    sell = 0
                    if 'sell' in entry.keys():
                        sell = entry['sell']
                    units = 0
                    if 'unit' in entry.keys():
                        try:
                            units = supported_units[entry['unit'].lower()]
                        except KeyError:
                            raise RuntimeError("UtilityRateDatabase error: unrecognized unit in rate structure")
                    mat.append((i + 1, j + 1, tier_max, units, rate, sell))
            urdb_data[data_name] = mat

    def try_get_demand_structure(urdb_name, data_name):
        mat = []
        if urdb_name in urdb_response.keys():
            structure = urdb_response[urdb_name]
            for i, period in enumerate(structure):
                for j, entry in enumerate(period):
                    rate = entry['rate']
                    if 'adj' in entry.keys():
                        rate += entry['adj']
                    tier_max = 1e38
                    if 'max' in entry.keys():
                        tier_max = entry['max']
                    if 'unit' in entry.keys():
                        if entry['unit'].lower() != "kW".lower():
                            raise RuntimeError("UtilityRateDatabase error: unrecognized unit in rate structure")
                    mat.append((i + 1, j + 1, tier_max, rate))
            if data_name:
                urdb_data[data_name] = mat
            else:
                return mat

    if "dgrules" in urdb_response.keys():
        rules = urdb_response['dgrules']  # dgrules
        if rules == "Net Metering":
            urdb_data['ur_metering_option'] = 0
        elif rules == "Net Billing Instantaneous":
            urdb_data['ur_metering_option'] = 2
        elif rules == "Net Billing Hourly":
            urdb_data['ur_metering_option'] = 3
        elif rules == "Buy All Sell All":
            urdb_data['ur_metering_option'] = 4
    else:
        # if no metering option provided, assume Net Metering
        urdb_data['ur_metering_option'] = 0

    if 'fixedchargefirstmeter' in urdb_response.keys() and 'fixedchargeunits' in urdb_response.keys():
        fixed_charge = urdb_response['fixedchargefirstmeter']
        fixed_charge_units = urdb_response['fixedchargeunits']
        if fixed_charge_units == "$/day":
            fixed_charge *= 365 / 12
        elif fixed_charge_units == "$/year":
            fixed_charge /= 12
        urdb_data['ur_monthly_fixed_charge'] = fixed_charge

    if 'mincharge' in urdb_response.keys():
        min_charge = urdb_response['mincharge']
        min_charge_units = urdb_response['minchargeunits']
        if min_charge_units == "$/year":
            urdb_data['ur_annual_min_charge'] = min_charge
        else:
            if min_charge_units == "$/day":
                min_charge *= 365 / 12
            urdb_data['ur_monthly_min_charge'] = min_charge

    try_get_schedule('energyweekdayschedule', 'ur_ec_sched_weekday')
    try_get_schedule('energyweekendschedule', 'ur_ec_sched_weekend')
    try_get_rate_structure('energyratestructure', 'ur_ec_tou_mat')

    try_get_demand_structure('demandratestructure', 'ur_dc_tou_mat')
    try_get_schedule('demandweekdayschedule', 'ur_dc_sched_weekday')
    try_get_schedule('demandweekendschedule', 'ur_dc_sched_weekend')

    flat_demand_structure = try_get_demand_structure('flatdemandstructure', None)

    if 'flatdemandmonths' in urdb_response.keys():
        urdb_data['ur_dc_enable'] = 1
        flat_mat = []
        flat_demand = urdb_response['flatdemandmonths']
        for month, period in enumerate(flat_demand):
            tiers = []
            for r in flat_demand_structure:
                if r[0] == int(period + 1):
                    tiers.append(r)
                    
            if len(tiers) == 0:
                raise ValueError("flatdemandstructure missing period number ", period)
            for t in tiers:
                month_row = []
                month_row.append(month)
                month_row += [t[i] for i in (1, 2, 3)]
                flat_mat.append(month_row)
        urdb_data['ur_dc_flat_mat'] = flat_mat
    # Fill out an empty flat rate structure if the rate has TOU demand but not flat demand    
    elif "demandratestructure" in urdb_response.keys():
        urdb_data['ur_dc_enable'] = 1
        # Enumerate a dc_flat table with $0/kW in 12 months
        flat_mat = []
        for i in range(0, 12):
            month_mat = [i, 1, 1e38, 0]
            flat_mat.append(month_mat)
        urdb_data['ur_dc_flat_mat'] = flat_mat
    else:
        urdb_data['ur_dc_enable'] = 0

    if urdb_data['ur_dc_enable'] == 1 and "ur_dc_tou_mat" not in urdb_data.keys():
        urdb_data['ur_dc_tou_mat'] = [[1, 1, 1e38, 0], ]
        urdb_data['ur_dc_sched_weekday'] = [[1] * 24 for i in range(12)]
        urdb_data['ur_dc_sched_weekend'] = urdb_data['ur_dc_sched_weekday']

    has_billing_demand = False

    lookback_range = 0
    if 'lookbackrange' in urdb_response.keys():
        lookback_range = urdb_response['lookbackrange']
        urdb_data['ur_billing_demand_lookback_period'] = lookback_range 
    
    lbp = 0
    if 'lookbackpercent' in urdb_response.keys():
        lbp = urdb_response['lookbackpercent'] * 100.0
        
        # Some demand ratchets apply in every month - use this as the default if lookbackmonths is not specified
        lbm = [True] * 12
        if 'lookbackmonths' in urdb_response.keys():
            lbm = urdb_response['lookbackmonths']
        
        lookback_percentages = [ [0, 0] ] * 12
        for i in range(0, len(lookback_percentages)):
            if lbm[i]:
                lookback_percentages[i][0] = lbp

        urdb_data['ur_billing_demand_lookback_percentages'] = lookback_percentages
    
    has_billing_demand = lookback_range > 0 or lbp > 0

    if has_billing_demand:
        urdb_data['ur_enable_billing_demand'] = True
        # Handle variables that aren't currently in URDB but are in SAM
        urdb_data['ur_billing_demand_minimum'] = 0

        dc_tou = urdb_data['ur_dc_tou_mat']
        max_period = 0
        for i in range(0, len(dc_tou)):
            if dc_tou[i][0] > max_period:
                max_period = dc_tou[i][0] 
        
        bd_periods = []
        for i in range(0, max_period):
            bd_periods.append([i+1, 1])
        
        urdb_data['ur_dc_billing_demand_periods'] = bd_periods

        print("Billing Demand Notice.\nThis rate includes billing demand adjustments and/or demand ratchets that may not be accurately reflected in the data downloaded from the URDB. Please check the information in the Description under Description and Applicability and review the rate sheet to be sure the billing demand inputs are correct.")
    else:
        urdb_data['ur_enable_billing_demand'] = False

    return urdb_data


"""
This example script shows how to set up the utility rate module using a rate downloaded from the Utility Rate

This example requires an NREL developer key, which can be obtained from https://developer.nrel.gov/signup/

Additional financial models, inputs, and outputs can be found at:
* PV: https://nrel-pysam.readthedocs.io/en/master/modules/Utilityrate5.html

Most recently tested against PySAM 4.2.0
Requires: nrel-pysam, requests

@author: brtietz
"""

# Download rate from URDB and save as file. If rate has already been downloaded, use file
def get_urdb_rate_data(page, key):
    """
    Using page value and user API key, returns json file containing specified tariff dictionary
    
    page: 'str' : required key for URDB website for specific eia_id tariff 
    key: 'str' : User specific API key for URDB website
    """
    
    # Full API can be viewed at: https://openei.org/services/doc/rest/util_rates/?version=8
    urdb_url = 'https://api.openei.org/utility_rates?version=8&format=json&detail=full&limit=3'
    get_url = urdb_url + '&api_key={api_key}&getpage={page_id}'.format(api_key=key, page_id=page)


    filename = "urdb_rate_{}.json".format(page)

    if not os.path.isfile(filename):
        print(get_url)
        CA = certifi.where()
        resp = requests.get(get_url, verify=CA)
    
        data = resp.json()
        
        # Cache rate as file
        if "error" not in data:
            with open(filename, 'w') as f:
                f.write(json.dumps(data, sort_keys=True, indent=2, separators=(',', ': ')))
    else:
        with open(filename, 'r') as f:
            data = json.load(f)

    return data


## Code to pull URDB tariff and append it to relevant agents ##
Ignore the warnings that result from this code block, the rest of the code will work as expected

In [ ]:
# Function to automatically grab each tariff dictionary from URDB website 
def get_tariff_dict(page_list, api_key):
    """
    Takes in list of relevant page labels for URDB site, returns tariff dictionary list that matches
    order of relevant FZ
    
    page_list : list : list of strings for looking up and retrieving tariff dictionaries of interest
    api_key : str : user specific API key for URDB website
    """
    # Empty array for storing returned tariff dictionaries
    tar = []
    
    # Loop through eag page in the page_list
    for page in page_list:
        # Get tariff dictionary
        urdb_response = get_urdb_rate_data(page, api_key)
        
        # Process tariff dictionary
        rates = URDBv8_to_ElectricityRates(urdb_response["items"][0]) 
        
        # Append tariff dictionary to empty list
        tar.append(rates)

    return tar

key = "tVEWe4twhwfTHpu58OTtTJ43Zih6o00JD4KJ2b4o" # Insert your API key here

# Access all page labels for URDB
page_list = df['Page/Label']

# Pull tariff_dict values
tar = 
(list(page_list), key)

# Assign tariff_dict values to new column
for i in range(len(tar)):
    t_dict = tar[i].copy()
    df['tariff_dict'].iloc[i] = [t_dict.copy()] #[tar[i].copy()]

# Verify tariff_dict has been assigned correctly
df['tariff_dict']

# Assign new URDB rates to the specified EIA IDs from the original agent pickle file #

In [ ]:
for i in range(len(df)):
    row = df.iloc[i]

    old_eia = res.loc[(res['eia_id']==str(row['eia_id']))&(res['tariff_name']==row['tariff_name'])]
    print(old_eia.index)
    new_t_name = row['tariff_name']
    # print(row['tariff_dict'])
    print(new_t_name)

    if len(old_eia) == 1:

        idx = old_eia.index
        res.loc[idx, 'tariff_dict'] = [row['tariff_dict']].copy()
        res.loc[idx, 'tariff_name'] = new_t_name
    
    elif len(old_eia) > 1:

        # Get list of all indexes in agent that match the update csv file
        idx = old_eia.index
        
        # For each index/row, update corresponding update with new tariff info 
        for j in idx:
            # print(j)
            idx = old_eia.index
            res.loc[idx, 'tariff_name'] = new_t_name
            res.loc[idx, 'tariff_dict'] = [row['tariff_dict']].copy()
            
    
    else:
        pass

print(res['tariff_dict'])

### Validate new code has correct updates and then save file ###
ensure the new tariff dict type is saving as type "dict". If they are not, let me know so I can send follow up code

In [ ]:
t_name = res.loc[(res['eia_id']=='40428')&(res['tariff_name']=='Schedule N - Navy Service'), 'tariff_name'].iloc[0]
t_dict = res.loc[res['eia_id']=='40428', 'tariff_dict'].iloc[0]
print(t_name)
print(t_dict)
for i in range(len(t_dict)):
    print(type(t_dict))
print(res['tariff_dict'])

In [ ]:
# Create new agent pickle file to use in model, will save in whatever the current directory you run this notebook in
res.to_pickle('3_18_trial_com_agents.pkl')

In [ ]:
res['load_kwh_in_bin_initial']
